Notebook para poder validar la transferencia correcta de la data comparando valores aleatorios de la data en RAW DE SNOWFLAKE y CLEAN DE SNOWFLAKE.
Se compara la tabla de products, instacart_orders y order_products. Solo se verifican los ids debido a la modelación y transformaciones de RAW a CLEAN.

In [10]:
import random   
import snowflake.connector
from os import getenv


In [11]:
def compare_data(table_name, id_column_name, no_comparisons, rango,snowflakecursor, snowflakecursor2, comparator,table_name2):
    ids=[]
    for _ in range(no_comparisons):
        id = random.randint(1, rango)
        ids.append(id)
    for id in ids:
        snowflakecursor.execute("SELECT %s FROM %s WHERE %s = %%s" % (comparator, table_name, id_column_name), (id,))
        snowresult = snowflakecursor.fetchall()
        snowflakecursor2.execute("SELECT %s FROM %s WHERE %s = %%s" % (comparator, table_name2, id_column_name), (id,))
        snowresult2 = snowflakecursor2.fetchall()
        if snowresult != snowresult2:
            print("Data mismatch for id "+str(id))
            print("RAW: "+str(snowresult))
            print("CLEAN: "+str(snowresult2))
            return False
    return True
        
    

In [12]:
config_snowflake_1 = {
    'user': getenv('SNOW_USER'),
    'password': getenv('SNOW_PASSWORD'),
    'account': getenv('SNOW_ACCOUNT'),
    'schema': 'RAW',
    'database': 'INSTACART_DB',}
config_snowflake_2 = {
    'user': getenv('SNOW_USER'),
    'password': getenv('SNOW_PASSWORD'),
    'account': getenv('SNOW_ACCOUNT'),
    'schema': 'CLEAN',
    'database': 'INSTACART_DB',}

snowflakecnx = snowflake.connector.connect(**config_snowflake_1)
snowflakecursor = snowflakecnx.cursor()
snowflakecnx2 = snowflake.connector.connect(**config_snowflake_2)
snowflakecursor2 = snowflakecnx2.cursor()
    

In [13]:
#Consultar tabla products en Snowflake para validar la conexión
snowflakecursor.execute("SELECT * FROM products LIMIT 20")
for x in snowflakecursor:
    print(x)

(1, 'Chocolate Sandwich Cookies', 61, 19)
(2, 'All-Seasons Salt', 104, 13)
(3, 'Robust Golden Unsweetened Oolong Tea', 94, 7)
(4, 'Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce', 38, 1)
(5, 'Green Chile Anytime Sauce', 5, 13)
(6, 'Dry Nose Oil', 11, 11)
(7, 'Pure Coconut Water With Orange', 98, 7)
(8, "Cut Russet Potatoes Steam N' Mash", 116, 1)
(9, 'Light Strawberry Blueberry Yogurt', 120, 16)
(10, 'Sparkling Orange Juice & Prickly Pear Beverage', 115, 7)
(11, 'Peach Mango Juice', 31, 7)
(12, 'Chocolate Fudge Layer Cake', 119, 1)
(13, 'Saline Nasal Mist', 11, 11)
(14, 'Fresh Scent Dishwasher Cleaner', 74, 17)
(15, 'Overnight Diapers Size 6', 56, 18)
(16, 'Mint Chocolate Flavored Syrup', 103, 19)
(17, 'Rendered Duck Fat', 35, 12)
(18, 'Pizza for One Suprema  Frozen Pizza', 79, 1)
(19, 'Gluten Free Quinoa Three Cheese & Mushroom Blend', 63, 9)
(20, 'Pomegranate Cranberry & Aloe Vera Enrich Drink', 98, 7)


In [14]:
#Consultar tabla dim_products en Snowflake para validar la conexión
snowflakecursor2.execute("SELECT * FROM dim_producto LIMIT 20")
for x in snowflakecursor2:
    print(x)

(1, 'Chocolate Sandwich Cookies', 'cookies cakes', 'snacks')
(2, 'All-Seasons Salt', 'spices seasonings', 'pantry')
(3, 'Robust Golden Unsweetened Oolong Tea', 'tea', 'beverages')
(4, 'Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce', 'frozen meals', 'frozen')
(5, 'Green Chile Anytime Sauce', 'marinades meat preparation', 'pantry')
(6, 'Dry Nose Oil', 'cold flu allergy', 'personal care')
(7, 'Pure Coconut Water With Orange', 'juice nectars', 'beverages')
(8, "Cut Russet Potatoes Steam N' Mash", 'frozen produce', 'frozen')
(9, 'Light Strawberry Blueberry Yogurt', 'yogurt', 'dairy eggs')
(10, 'Sparkling Orange Juice & Prickly Pear Beverage', 'water seltzer sparkling water', 'beverages')
(11, 'Peach Mango Juice', 'refrigerated', 'beverages')
(12, 'Chocolate Fudge Layer Cake', 'frozen dessert', 'frozen')
(13, 'Saline Nasal Mist', 'cold flu allergy', 'personal care')
(14, 'Fresh Scent Dishwasher Cleaner', 'dish detergents', 'household')
(15, 'Overnight Diapers Size 6', 'di

In [15]:
#Ver la cantidad de registros de cada tabla y comparar con su modelo en RAW
#tabla dim_products en CLEAN con tabla products en RAW
snowflakecursor.execute("SELECT COUNT(*) FROM products")
for x in snowflakecursor:
    no_rows = x[0]
    print("Cantidad de registros en tabla products en RAW: "+str(no_rows))
snowflakecursor2.execute("SELECT COUNT(*) FROM dim_producto")
for x in snowflakecursor2:
    no_rows = x[0]
    print("Cantidad de registros en tabla dim_producto en CLEAN: "+str(no_rows))

Cantidad de registros en tabla products en RAW: 49694
Cantidad de registros en tabla dim_producto en CLEAN: 49694


In [16]:
#tabla fct_orders en CLEAN con tabla instacart_orders en RAW
snowflakecursor.execute("SELECT COUNT(*) FROM instacart_orders")
for x in snowflakecursor:
    no_rows = x[0]
    print("Cantidad de registros en tabla instacart_orders en RAW: "+str(no_rows))
snowflakecursor2.execute("SELECT COUNT(*) FROM fct_orders")
for x in snowflakecursor2:
    no_rows = x[0]
    print("Cantidad de registros en tabla fct_orders en CLEAN: "+str(no_rows))

Cantidad de registros en tabla instacart_orders en RAW: 478967
Cantidad de registros en tabla fct_orders en CLEAN: 478952


In [17]:
#tabla fct_orders_producto en CLEAN con tabla order_products en RAW
snowflakecursor.execute("SELECT COUNT(*) FROM order_products")
for x in snowflakecursor:
    no_rows = x[0]
    print("Cantidad de registros en tabla order_products en RAW: "+str(no_rows))
snowflakecursor2.execute("SELECT COUNT(*) FROM fct_orders_producto")
for x in snowflakecursor2:
    no_rows = x[0]
    print("Cantidad de registros en tabla fct_orders_producto en CLEAN: "+str(no_rows))

Cantidad de registros en tabla order_products en RAW: 4545007
Cantidad de registros en tabla fct_orders_producto en CLEAN: 4545007


In [18]:
# Tabla products con tabla dim_products, 49694 filas
print("Comparing id_product: products vs dim_producto")
print("Data is consistent" if compare_data("products", "product_id", 100, 49694, snowflakecursor, snowflakecursor2, "product_name", "dim_producto")
      else "Data is inconsistent")
print()
# Tabla instacart_orders con tabla fct_orders, 478952 filas de order_id unico
print("Comparing id_order: instacart_orders vs fct_orders")
print("Data is consistent" if compare_data("instacart_orders", "order_id", 100, 478967, snowflakecursor, snowflakecursor2, "order_number", "fct_orders")
      else "Data is inconsistent")
print()
# Tabla order_products con tabla fct_orders_producto, 4545007 filas
print("Comparing id_order: order_products vs fct_orders_products")
print("Data is consistent" if compare_data("order_products", "order_id", 100, 4545007, snowflakecursor, snowflakecursor2, "product_id", "fct_orders_producto")
      else "Data is inconsistent")

Comparing id_product: products vs dim_producto
Data is consistent

Comparing id_order: instacart_orders vs fct_orders
Data is consistent

Comparing id_order: order_products vs fct_orders_products
Data is consistent
